In [1]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape
import pandas as pd
import os

In [2]:
def fetch(url):
    """
    매개변수로 전달받을 url을 기반으로 웹 페이지를 추출
    웹 페이지의 Content-Type 헤더를 통해 인코딩 형식 확인
    반환값: str 자료형의 HTML
    """
    f = urlopen(url)
    # HTTP 헤더를 기반으로 인코딩 형식 추출
    encoding = f.info().get_content_charset(failobj="utf-8")
    # 추출한 인코딩 형식을 기반으로 문자열 디코딩
    html = f.read().decode(encoding)
    return html

In [3]:
def scrape(html):
    """
    매개변수 html로 받은 HTML을 기반으로 정규 표현식을 사용해 도서 정보를 추출합니다.
    반환값: 도서(dict) 리스트
    """
    books = []
    # re.findall()을 사용해 도서 하나에 해당하는 HTML을 추출
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
        # 도서의 URL을 추출
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        # 태그를 제거해서 도서의 제목 추출
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append(pd.DataFrame({'url': [url], 'title': [title]}))
    return pd.concat(books)

In [4]:
def save(db_path, books):
    with sqlite3.connect(os.path.join('.', db_path)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            books.to_sql(name = 'BOOKS_INFO', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
    
    query = 'SELECT * FROM BOOKS_INFO'
    df = pd.read_sql(query, con = con)
    return df

In [5]:
html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [6]:
df = scrape(html)
df.reset_index(drop=True, inplace=True)
df2 = save('books.db', df)
df2

,url,title
0,http://www.hanbit.co.kr/store/books/look.php?p...,최신 관리회계
1,http://www.hanbit.co.kr/store/books/look.php?p...,리눅스 입문자를 위한 명령어 사전
2,http://www.hanbit.co.kr/store/books/look.php?p...,파타고니아 이야기
3,http://www.hanbit.co.kr/store/books/look.php?p...,"풀스택 서버리스 : 리액트, AWS, 그래프QL을 이용한 최신 애플리케이션 개발"
4,http://www.hanbit.co.kr/store/books/look.php?p...,한 권으로 배우는 작고 예쁜 꽃자수
5,http://www.hanbit.co.kr/store/books/look.php?p...,"IT CookBook, 처음 만나는 회로이론(2판)"
6,http://www.hanbit.co.kr/store/books/look.php?p...,안전필수 시스템 제어 설계
7,http://www.hanbit.co.kr/store/books/look.php?p...,러닝 리액트(2판)
8,http://www.hanbit.co.kr/store/books/look.php?p...,업무에 바로 쓰는 SQL 튜닝
9,http://www.hanbit.co.kr/store/books/look.php?p...,데이터 스토리
